# Seleção de Features

A **seleção de features** desempenha um papel crucial em projetos de IA, oferecendo uma maneira eficaz de **melhorar a precisão e a eficiência dos modelos**. 

Ao identificar e selecionar as features mais relevantes e informativas para um problema específico, os cientistas de dados podem:

- **Reduzir a dimensionalidade dos dados**  
- **Mitigar o impacto de features irrelevantes ou redundantes**  
- **Melhorar a capacidade de generalização dos modelos**

Além disso, a seleção de features também pode contribuir significativamente para a **interpretabilidade dos modelos**, permitindo aos stakeholders compreender melhor os fatores que influenciam as decisões do sistema de IA e aumentar a confiança nas previsões e insights gerados.

## Filtros Estatísticos
Essas técnicas avaliam as características com base em estatísticas como **correlação**, **teste de qui-quadrado**, **análise de variância (ANOVA)** ou **informações mútuas**, e selecionam as características mais relevantes de acordo com essas métricas.

## Wrapper Methods
Estas técnicas envolvem a avaliação de subconjuntos de características com base no desempenho de um modelo preditivo.  
Exemplos incluem:
- **Recursive Feature Elimination (RFE)**
- **Sequential Feature Selection (SFS)**
- **Genetic Algorithms**

## Embedded Methods
Estas técnicas incorporam a seleção de características diretamente no processo de treinamento do modelo.  
Algoritmos de aprendizado de máquina como **árvores de decisão** e modelos lineares com **regularização** (por exemplo, **Lasso** e **Ridge**) incorporam naturalmente algum tipo de seleção de características.

## Redução de Dimensionalidade
Técnicas como **Análise de Componentes Principais (PCA)** e **T-Distributed Stochastic Neighbor Embedding (t-SNE)** reduzem a dimensionalidade dos dados ao criar representações compactas das características originais.

## Seleção de Características Baseada em Importância de Modelo
Alguns modelos de aprendizado de máquina fornecem uma medida de importância para cada característica.  
Métodos como **SelectFromModel** aproveitam essas informações para selecionar as características mais importantes.

## Binning/Discretização
Em alguns casos, pode ser útil transformar variáveis numéricas em variáveis categóricas dividindo-as em intervalos (ou *bins*).  
Isso pode ajudar a capturar **padrões não lineares** e **reduzir a sensibilidade a outliers**.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score

### Carregar e visualizar os dados - Regressão

In [2]:
# carregar os dados - regressao
df = pd.read_csv('./datasets/dataset_colesterol.csv')

In [3]:
# visualizar estrutura dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     996 non-null    object 
 2   Fumante             997 non-null    object 
 3   Nível de Atividade  996 non-null    object 
 4   Idade               997 non-null    float64
 5   Peso                997 non-null    float64
 6   Altura              997 non-null    float64
 7   Colesterol          1000 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 62.6+ KB


In [4]:
# visualizar dataframe
df.head(10)

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33.0,85.1,186.0,199.63
1,2,A,Não,Moderado,68.0,105.0,184.0,236.98
2,3,O,Não,Alto,25.0,64.8,180.0,161.79
3,4,A,Não,Alto,43.0,120.2,167.0,336.24
4,5,AB,Não,Baixo,79.0,88.5,175.0,226.23
5,6,B,Não,Baixo,68.0,66.8,170.0,185.31
6,7,A,Sim,Baixo,60.0,117.3,181.0,289.33
7,8,O,Sim,Moderado,35.0,86.9,174.0,216.48
8,9,O,Não,Baixo,62.0,81.3,166.0,235.30
9,10,B,Sim,Alto,44.0,32.7,165.0,97.79


In [5]:
# ajustar dataframe
df.drop('Id', axis=1, inplace=True)

# aplicar onehotencoding nas variaveis categoricas
df = pd.get_dummies(df, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

In [6]:
df = df.dropna()

In [7]:
# dataframe atualizado
df

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33.0,85.1,186.0,199.63,False,False,True,False,False,True,False,True,False
1,68.0,105.0,184.0,236.98,True,False,False,False,True,False,False,False,True
2,25.0,64.8,180.0,161.79,False,False,False,True,True,False,True,False,False
3,43.0,120.2,167.0,336.24,True,False,False,False,True,False,True,False,False
4,79.0,88.5,175.0,226.23,False,True,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31.0,68.1,166.0,206.81,False,True,False,False,False,True,False,False,True
996,51.0,47.7,170.0,128.03,False,False,False,True,True,False,True,False,False
997,39.0,85.5,176.0,211.14,False,True,False,False,True,False,False,True,False
998,61.0,91.2,161.0,284.53,False,True,False,False,False,True,False,True,False


### Treinar modelo de regressão linear múltipla com RFE

In [8]:
# separar o X e y
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [9]:
# separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [10]:
# treinar o modelo com rfe
# RFE (Recursive Feature Elimination)
# uso um estimador e defino uma quantidade de features (dois hiperparametros)
# o RFE faz iteracoes iniciando com todas as features e eliminando a cada iteracao ate atingir a quantidade definida
# elimina caracteristicas/features menos importantes
rfe_method = RFE(estimator=LinearRegression(), n_features_to_select=6)
rfe_method.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=6)

In [11]:
# Quais features foram selecionadas
X_train.columns[(rfe_method.get_support())]

Index(['Peso', 'Altura', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Baixo', 'Nível de Atividade_Moderado'],
      dtype='object')

In [12]:
# ranking de features
def mostrar_ranking(metodo_fs, X_train):

    # obter o ranking de features
    ranking = rfe_method.ranking_

    # obter os nomes das features
    nomes_features = X_train.columns.to_list()

    # criar um dataframe com os rankings e nomes das features
    df_ranking = pd.DataFrame({'Features': nomes_features, 'Ranking': ranking})

    # ordenar o dataframe pelo ranking
    df_ranking = df_ranking.sort_values(by='Ranking')

    # exibir ranking
    print(df_ranking)

In [13]:
# ranking de features do RFE regressao
mostrar_ranking(rfe_method, X_train)

                       Features  Ranking
1                          Peso        1
2                        Altura        1
7                   Fumante_Não        1
8                   Fumante_Sim        1
10     Nível de Atividade_Baixo        1
11  Nível de Atividade_Moderado        1
9       Nível de Atividade_Alto        2
3             Grupo Sanguíneo_A        3
5             Grupo Sanguíneo_B        4
6             Grupo Sanguíneo_O        5
4            Grupo Sanguíneo_AB        6
0                         Idade        7


In [14]:
# função para avaliar performance
def performance_regressao(modelo, X_test, y_test):

    # fazer predicao com o modelo no conjunto de testes
    y_pred = modelo.predict(X_test)

    # calcular MSE
    mse = mean_squared_error(y_test, y_pred)

    # calcular RMSE manualmente
    rmse = np.sqrt(mse)

    return rmse

In [15]:
# performance regressao com RFE
performance_regressao(rfe_method, X_test, y_test)

np.float64(8.902221594870712)

### Treinar modelo sem RFE

In [16]:
# treinar modelo de regressao sem RFE
model_reg = LinearRegression()
model_reg.fit(X_train, y_train)

LinearRegression()

In [17]:
# performance regressao com RFE
performance_regressao(model_reg, X_test, y_test)

np.float64(8.836354472209106)

### Treinar Modelo de Regressão Linear com RFECV

In [19]:
rfe_method_cv = RFECV(estimator=LinearRegression(), min_features_to_select=6, cv=5)
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LinearRegression(), min_features_to_select=6)

In [20]:
performance_regressao(rfe_method_cv, X_test, y_test)

np.float64(8.846226800088205)

In [21]:
# quais features foram escolhidas
X_train.columns[(rfe_method_cv.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_A', 'Grupo Sanguíneo_AB',
       'Grupo Sanguíneo_B', 'Grupo Sanguíneo_O', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto', 'Nível de Atividade_Baixo',
       'Nível de Atividade_Moderado'],
      dtype='object')

In [22]:
# quantas features foram selecionadas
rfe_method_cv.n_features_

np.int64(11)

### Treinar modelo de regressão com SelectFromModel

In [30]:
sfm_method = SelectFromModel(estimator=model_reg, max_features=4, threshold=0.5)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression(), max_features=4, threshold=0.5)

In [32]:
# quais features foram escolhidas
X_train.columns[(sfm_method.get_support())]

Index(['Fumante_Não', 'Fumante_Sim', 'Nível de Atividade_Baixo',
       'Nível de Atividade_Moderado'],
      dtype='object')

In [33]:
# treinar modelo com as features selecionadas
X_train_ajustado_reg = sfm_method.transform(X_train)
X_test_ajustado_reg = sfm_method.transform(X_test)
model_reg.fit(X_train_ajustado_reg, y_train)


LinearRegression()

In [34]:
# performance do modelo com SelectFromModel
performance_regressao(model_reg, X_test_ajustado_reg, y_test)

np.float64(45.148296339784416)

### Carregar e preparar dados - Classificação

In [35]:
# Carregar o dataset
df2 = pd.read_csv('./datasets/fruit_quality.csv')

In [36]:
# Visualizar estrutura
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   A_id         4000 non-null   int64  
 1   Size         4000 non-null   float64
 2   Weight       4000 non-null   float64
 3   Sweetness    4000 non-null   float64
 4   Crunchiness  4000 non-null   float64
 5   Juiciness    4000 non-null   float64
 6   Ripeness     4000 non-null   float64
 7   Acidity      4000 non-null   float64
 8   Quality      4000 non-null   object 
dtypes: float64(7), int64(1), object(1)
memory usage: 281.4+ KB


In [37]:
# visualizar df
df.head(10)

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33.0,85.1,186.0,199.63,False,False,True,False,False,True,False,True,False
1,68.0,105.0,184.0,236.98,True,False,False,False,True,False,False,False,True
2,25.0,64.8,180.0,161.79,False,False,False,True,True,False,True,False,False
3,43.0,120.2,167.0,336.24,True,False,False,False,True,False,True,False,False
4,79.0,88.5,175.0,226.23,False,True,False,False,True,False,False,True,False
5,68.0,66.8,170.0,185.31,False,False,True,False,True,False,False,True,False
6,60.0,117.3,181.0,289.33,True,False,False,False,False,True,False,True,False
7,35.0,86.9,174.0,216.48,False,False,False,True,False,True,False,False,True
8,62.0,81.3,166.0,235.30,False,False,False,True,True,False,False,True,False
9,44.0,32.7,165.0,97.79,False,False,True,False,False,True,True,False,False


In [38]:
# Ajustar Dataframe

# Remover a coluna 'A_id'
df2.drop('A_id', axis=1, inplace=True)

# Transformar a coluna 'Quality' em 0 ou 1 (binária)
df2['Quality'] = (df2['Quality'] == 'good').astype(int)

# Visualizar resultado
df2


,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,1
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,1
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,0
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,1
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,1
...,...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,0
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,1
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,0
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,1


### Treinar modelo de regressão logistica com RFE

In [39]:
# separar o X e y
X = df2.drop('Quality', axis=1)
y = df2['Quality']

In [40]:
# separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [58]:
# treinar o modelo com rfe
# RFE (Recursive Feature Elimination)
# uso um estimador e defino uma quantidade de features (dois hiperparametros)
# o RFE faz iteracoes iniciando com todas as features e eliminando a cada iteracao ate atingir a quantidade definida
# elimina caracteristicas/features menos importantes
rfe_method = RFE(estimator=LogisticRegression(), n_features_to_select=5)
rfe_method.fit(X_train, y_train)

RFE(estimator=LogisticRegression(), n_features_to_select=5)

In [59]:
# Quais features foram selecionadas
X_train.columns[(rfe_method.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Acidity'], dtype='object')

In [60]:
# ranking de features do RFE regressao
mostrar_ranking(rfe_method, X_train)

      Features  Ranking
0         Size        1
1       Weight        1
2    Sweetness        1
4    Juiciness        1
6      Acidity        1
5     Ripeness        2
3  Crunchiness        3


In [61]:
# função para avaliar performance
def performance_classificacao(modelo, X_test, y_test):

    # fazer predicao com o modelo no conjunto de testes
    y_pred = modelo.predict(X_test)

    # avaliar desempenho
    return f1_score(y_test, y_pred)

In [62]:
# performance regressao com RFE
performance_classificacao(rfe_method, X_test, y_test)

0.7738193869096934

### Treinar modelo sem RFE

In [56]:
# treinar modelo sem rfe
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

LogisticRegression()

In [57]:
# validar performance
performance_classificacao(model_lr, X_test, y_test)

0.7787903893951947

### Treinar Modelo de Regressão Logistica com RFECV

In [67]:
rfe_method_cv = RFECV(estimator=LogisticRegression(), min_features_to_select=4, cv=5, scoring='f1_weighted')
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LogisticRegression(), min_features_to_select=4,
      scoring='f1_weighted')

In [68]:
performance_regressao(rfe_method_cv, X_test, y_test)

np.float64(0.473462423711393)

In [69]:
# quais features foram escolhidas
X_train.columns[(rfe_method_cv.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Ripeness', 'Acidity'], dtype='object')

In [70]:
# quantas features foram selecionadas
rfe_method_cv.n_features_

np.int64(6)

### Treinar modelo de regressão logística com SelectFromModel

In [90]:
sfm_method = SelectFromModel(estimator=model_lr, max_features=5, threshold=0.01)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LogisticRegression(), max_features=5, threshold=0.01)

In [91]:
# quais features foram escolhidas
X_train.columns[(sfm_method.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Acidity'], dtype='object')

In [92]:
# treinar modelo com as features selecionadas
X_train_ajustado_class = sfm_method.transform(X_train)
X_test_ajustado_class = sfm_method.transform(X_test)
model_lr.fit(X_train_ajustado_class, y_train)


LogisticRegression()

In [96]:
performance_classificacao(model_lr, X_test_ajustado_class, y_test)

0.7738193869096934